In [ ]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm

In [ ]:
data = pd.read_csv(".../4_weeks_without_missing_onset.csv")
data.isnull().sum()

In [ ]:
drug_data = pd.read_csv(".../MS_patients_meas3.csv")
#drug_data = drug_data[['patient_id','start_date','end_date','ms_first_line','ms_second_line','ms_other']]
drug_data = drug_data[~drug_data['patient_id'].str.contains("TUR_MS",na=False)]
drug_data['patient_id'] = drug_data['patient_id'].astype('int64')

In [ ]:
#For between two visit date
data['Date_of_visit'] = pd.to_datetime(data['Date_of_visit'],format ='%Y-%m-%d')
drug_data["start_date"] = pd.to_datetime(drug_data["start_date"],format ='%Y-%m-%d')
drug_data["end_date"] = pd.to_datetime(drug_data["end_date"],format ='%Y-%m-%d')


for index, row in data.iterrows():
    print(index)
    patient_postcode = row['CAP']
    patient_id = row['Patient_ID']
    visit_date = row['Date_of_visit']
    index_row = data.index[(data['Patient_ID'] == patient_id)&
                           (data['Date_of_visit'] == visit_date)].tolist()
    
    if len(data) == index+1:
        break
        
    elif data.iloc[index+1]["Patient_ID"] == patient_id:
        next_visit_date = data.loc[index+1,'Date_of_visit']
        
        if patient_id in list(drug_data["patient_id"]):
            
            matching_rows = drug_data[(drug_data['patient_id'] == patient_id) &
                                      (drug_data['start_date'].dt.date >= visit_date.date()) &
                                      (drug_data['end_date'].dt.date < next_visit_date.date())]
            
            
            if len(matching_rows) != 0:
                matching_rows = matching_rows.fillna(0)
                data.loc[index+1,"First_line_Treat"] = sum(matching_rows['ms_first_line'])
                data.loc[index+1,"Second_line_Treat"] = sum(matching_rows['ms_second_line'])
                data.loc[index+1,"Other_line_Treat"] = sum(matching_rows['ms_other'])
            else:
                print("matching row is zero")
        else:
            print("The patient id " + str(patient_id) + " cannot be found in weekly dataset")
    else:
        continue

data.head()

In [ ]:
# For 24 weeks before the visit date for first observation of visit to visit data
data['Date_of_visit'] = pd.to_datetime(data['Date_of_visit'],format ='%Y-%m-%d')
drug_data["start_date"] = pd.to_datetime(drug_data["start_date"],format ='%Y-%m-%d')
drug_data["end_date"] = pd.to_datetime(drug_data["end_date"],format ='%Y-%m-%d')
first_obs = data.groupby(["Patient_ID"]).nth(0).reset_index()

for index, row in data.iterrows():
    print(index)
    patient_postcode = row['CAP']
    patient_id = row['Patient_ID']
    visit_date = row['Date_of_visit']
    index_row = data.index[(data['Patient_ID'] == patient_id)&
                           (data['Date_of_visit'] == visit_date)].tolist()

        
    if patient_id in list(drug_data["patient_id"]):
        
        matching_rows = drug_data[(drug_data['patient_id'] == patient_id) &
                                  (drug_data['start_date'].dt.date >= visit_date.date() - 
                                   pd.DateOffset(weeks=4)) &
                                  (drug_data['end_date'].dt.date < visit_date.date())]
            
        if len(matching_rows) != 0:
            matching_rows = matching_rows.fillna(0)
            data.loc[index_row,"First_line_Treat"] = sum(matching_rows['ms_first_line'])
            data.loc[index_row,"Second_line_Treat"] = sum(matching_rows['ms_second_line'])
            data.loc[index_row,"Other_line_Treat"] = sum(matching_rows['ms_other'])
        else:
            print("matching row is zero")
    else:
        print("The patient id " + str(patient_id) + " cannot be found in weekly dataset")

data.head()